# Descarga de PDBs de la proteína CDK2
Esta libreta de Jupiter permite descargar las estructuras cristalográficas disponibles de una proteína dada una secuencia de UniProt, un valor de identidad y un valor de covertura.

Como resultado, al final de la ejecución todos los archivos pdb disponibles de la proteína son descargados, generándose tres archivos por cada pdb id:

- Archivo en crudo del pdb id (el archivo tal y como se puede descargar del PDB).
- Archivo pdb con únicamnete la cadena protéica correspondiente a la proteína de interés.
- Archivo pdb con todas las moléculas con la etiqueta HETATM que acompañana a la proteína pudiendo ser moléculas pequeñas, iones o residuos portéicos modificados (las moléculas d eagua son removidas).

In [3]:
from prody import *
from pylab import *
import pickle

In [1]:
import sys
# Añadimos la ruta del directorio raíz para tener acceso a los archivos y módulos
sys.path.append(r'..')

## Secuencia de la proteína
Se define el nombre de la proteína y su identificador en UNIPROT:

In [4]:
# Nombre de la proteina de interes
prot_name = 'cdk2'
# Secuencia P28482 (ERK2_HUMAN)
uniprot_id = "P24941"

Se obtiene la secuencia de la proteína y se guarda en un archivo en la carpeta data:

In [7]:
# Secuencia de la CDK2 de UniProt
import requests
from Bio import SeqIO

# Descargamos el fasta de UniProt
url_fasta = requests.get("https://www.uniprot.org/uniprot/" + uniprot_id + ".fasta")
file_name_fasta = '../data/' + uniprot_id + '.fasta'
open(file_name_fasta, 'wb').write(url_fasta.content)
# Leemos la secuenciade aminoácidos
fasta_prot = SeqIO.read(open(file_name_fasta),'fasta')
seq_prot = str(fasta_prot.seq)
print(seq_prot)
print(F'\nLa longitud de la secuancia es: {len(seq_prot)} residuos.')

MENFQKVEKIGEGTYGVVYKARNKLTGEVVALKKIRLDTETEGVPSTAIREISLLKELNHPNIVKLLDVIHTENKLYLVFEFLHQDLKKFMDASALTGIPLPLIKSYLFQLLQGLAFCHSHRVLHRDLKPQNLLINTEGAIKLADFGLARAFGVPVRTYTHEVVTLWYRAPEILLGCKYYSTAVDIWSLGCIFAEMVTRRALFPGDSEIDQLFRIFRTLGTPDEVVWPGVTSMPDYKPSFPKWARQDFSKVVPPLDEDGRSLLSQMLHYDPNKRISAKAALAHPFFQDVTKPVPHLRL

La longitud de la secuancia es: 298 residuos.


### BLASTp de la secuencia en el PDB
Descomentar el código para volver a ejecutar, o en su defecto, recuperar el archivo de blast guardado en el directorio.

In [8]:
# Hacemos un blast
blast_record = blastPDB(seq_prot)

# Lo guardamos para poder usarlo y no tener que rehacerlo
# pickle.dump(blast_record, open('B_DATOS/cdk2_blast_record.pkl', 'wb'))

# Para abrirlo:
blast_record = pickle.load(open('../data/cdk2_blast_record.pkl', 'rb'))

@> Blast searching NCBI PDB database for "MENFQ..."
@> Blast search completed in 9.3s.                  


Se obtienen los "hits" del BLAST según un valor de identidad especificado. También se obtiene una lista con los identificadores de dichos hits.

In [9]:
identidad  = 95
pdbids = blast_record.getHits(percent_identity = identidad) # Devuelve un diccionario con cada proteína hit
len(pdbids) # 391 proteínas tienen un 95% de identidad con la seucuencia
pdbids_list = list(pdbids.keys()) # lista de pdb_ids de las estructuras de CDK2
print(F'Se obtuvo un total de {len(pdbids_list)} hits con una identidad del {identidad}%')

Se obtuvo un total de 408 hits con una identidad del 95%


### Lista de identificadores de los PDB hits
Se guarda un archivo csv con todos los identificadores. Descomentar el código para volver a guardar.  

In [77]:
# Guarda la lista de PDB IDs de los hits
import csv
num_pdb_ids = len(pdbids_list); num_pdb_ids
with open(F'../data/CDK2_pdb_{str(num_pdb_ids)}_IDs_{uniprot_id}.csv', 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(pdbids_list)

## Selección de los residuos usados por Pisani (2016)
Para el caso de la proteína CDK2, vamos a utilizar una secuencia de residuos dada por Pisani, *et al.* (2016), la cual corresponde a la mayor parte de las regiones con estructura secundaria de la proteína en la región N-terminal de la misma.

In [7]:
from modulos.subsecuencias_cdk2 import get_pisani_residues

# Se obtiene la lista de residuos, incluida en formato de cadena de texto
pisiani_residues = get_pisani_residues('list')
pisiani_residues_str = get_pisani_residues('str')
open('../data/List_Pisani_residues.txt', 'w').write(pisiani_residues_str)
print(len(pisiani_residues), "residuos corresponden a la secuencia dada por Pisani.")

117 residuos corresponden a la secuencia dada por Pisani.


## Descarga de los archivos PDB
El siguiente ciclo descarga cada uno de los archivos PDB de la lista de identificadores PDB.
* Descarga el archivo PDB completo
* Descarga únicamente la cadena protéica del PDB con mayor identidad a la secuencia de UniProt dada (**Quedan alineadas por Pisani**)
* Descarga cualquier ligando asociado a la cadena protéica de mayor identidad.

In [ ]:
prot_name = 
# Directorios
DIR_OUT_RAW_PDBS = F'../../ARCHIVOS/CRISTALES/PDB_{prot_name}_RAW_files/'
DIR_OUT_PROT_CHAINS = F'../../ARCHIVOS/CRISTALES/PROT_{prot_name}_CHAINS/'
DIR_OUT_PROT_LIGS = F'../../ARCHIVOS/CRISTALES/LIGS_{prot_name}/RAW/'

# Crea los directorios si no existen
import os
for dir in [DIR_OUT_RAW_PDBS, DIR_OUT_PROT_CHAINS, DIR_OUT_PROT_LIGS]:
    if not os.path.exists(dir):
        os.makedirs(dir)

In [ ]:
# Estructura de referencia para alinear los PDBs, se usarán los CA de los átomos definidos por Pisani
best_id = blast_record.getBest()['pdb_id']
chain_best_id = pdbids[best_id]['chain_id']
ref_struct = parsePDB(best_id, 
                      folder = DIR_OUT_RAW_PDBS).select('protein and chain ' + 
                                                                 chain_best_id)

for pdb_id in pdbids_list:
    try:
        # Obtiene el id de la cadena correspondiente a la mejor estructura empatada según el BLAST
        chain_id = pdbids[pdb_id]['chain_id']
        # Obtiene del pdb la estructura y la guarda en el folder
        pdb_cry = parsePDB(pdb_id, folder = DIR_OUT_RAW_PDBS)
        # Selecciona de la proteína a la cadena que el BLAST indicó
        pdb_chain = pdb_cry.select('protein and chain ' + chain_id)
        # Realiza el alineamiento usando los residuos de Pisani
        pdb_alg = matchAlign(pdb_chain, ref_struct, 
                             overlap=85, tarsel=F'calpha and resnum {pisiani_residues_str}') 
        # Retorna un tuple con la estructura alineada en el primer índice
        protein = pdb_alg[0]
    except Exception as e:
        print(e, "Error al alinear:", pdb_id)
        continue
    else:
    repr(protein)
        if protein: # Si hubo una cadena, la guarda
            protein.setChids("A") # fuerza que la cadena sea renombrada a "A", para homogenizar
            writePDB(DIR_OUT_PROT_CHAINS + pdb_id + '_A.pdb', protein) # Guarda el archivo
        # Comprueba si existen ligandos (moléculas no protéicas)
        ligand = pdb_cry.select('hetatm')
        repr(ligand)
        if ligand: # Si hubo un ligando, lo guarda
            writePDB(DIR_OUT_PROT_LIGS + pdb_id + '_lig.pdb', ligand)

# ¡Terminado!